In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [6]:
#filename = "/home/viviane/GravLens/Scraping/masterlens/masterlens_44.html"  # list with dfs from tables
filename = '/home/vivianemm/PycharmProjects/GravLens/Database/Scraping/masterlens/masterlens_1.html'
df_list = pd.read_html(filename)

In [7]:
# important dfs: 5,6,7,11,12,13,14,16,17,18,19,20,21
for i in df_list:
    print(i.columns.values)

[0]
[0 1 2 3]
[0 1 2 3 4 5]
[0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1 2]
[0 1]
[0 1]
[0 1]
[0 1 2 3]
[0 1]
[0 1 2 3]
[0 1 2 3 4 5 6 7]
[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10]
[0 1 2 3 4 5]
[0 1 2 3 4 5 6 7 8]
[0 1]
[0 1]
[0 1 2 3 4 5 6 7 8]


### Defining functions

In [8]:
def pick_table(df_list, key):
    for df in df_list:
        list_labels = list(df.columns.values)
        if any(isinstance(i, tuple) for i in list_labels):  # if labels is nested list
            labels = [item for t in list_labels for item in t]  # flatten
        else:
            labels = list_labels

        list_values = df.values.tolist()
        if any(isinstance(i, list) for i in list_values):  # if values is nested list
            values = [item for t in list_values for item in t]  # flatten
        else:
            values = list_values
              
        if key in values or key in labels:
            #df_list.remove(df)
            return df

In [9]:
def drop_nans(df):
    df.dropna(axis=0, how='all', inplace=True)
    df.dropna(axis=1, how='all', inplace=True)
    return None

In [10]:
def split_col(df, old_new_list):  # old_new_list = [[oldname, newname], ...]
    for pair in old_new_list:
        if pair[0] in df.columns and type(list(df[pair[0]])[0]) == str and "±" in list(df[pair[0]])[0]:
            df[[pair[1], pair[1] + '_err']] = df[pair[0]].str.split("±", n = 1, expand = True)
            df.drop(pair[0], axis=1, inplace=True)
        else:
            pass
    return None

In [11]:
def get_table_links(table):
    href_list = []
    
    if isinstance(table, list):
        for row in table:
            href = row.find('a')['href']
            href_list.append(href)
    else:
        href_tags = table.find_all("a")
        for tag in href_tags:
            href = tag.get('href')
            href_list.append(href)
    return href_list

In [12]:
def save(df, filename, i=False):
    if len(df) > 0:
        df.to_csv("/home/viviane/GravLens/Scraping/results/" + filename, index = i)
        #df.to_csv('/home/vivianemm/PycharmProjects/GravLens/Scraping/results' + filename, index = i)
        
    return None

### System table
### (5, 6, 7, 11, 12, 20)

In [13]:
def system(df_list):
    # joining 5,6,7,11
    frames =[pick_table(df_list, 'Discovery'), pick_table(df_list, 'Lens Kind'), 
             pick_table(df_list,'Lens Grade'), pick_table(df_list, 'Description')]
    result_df = pd.concat(frames)
    result_df.reset_index(drop=True, inplace=True)
    system_df = result_df.drop(columns=0).T
    system_df.columns = list(result_df[0])
    drop_nans(system_df)
    
    # Splitting value ± error columns
    split_col(system_df, [['Einstein_R ["]','Einstein_R'], ['z_Lens', 'z_lens'], ['z_Source(s)', 'z_source'],
                          ['Stellar velocity disp', 'Stellar_v_disp']])
    
    # adding other 2 tables
    date_df = pick_table(df_list, 'Discovery Date:')
    n_df = pick_table(df_list, 'Number of Source Plane Images')
    
    system_df['Discovery Date'] = date_df[3][0]
    system_df['Name'] = date_df[1][0]
    system_df['N Images'] = n_df[1][0]
    
    return system_df

In [14]:
system_df = system(df_list)
system_df

,Discovery,Lens Kind,Lens Grade,Einstein_R quality,z_Lens quality,z_Source quality,Description,Einstein_R,Einstein_R_err,z_lens,z_lens_err,z_source,z_source_err,Stellar_v_disp,Stellar_v_disp_err,Discovery Date,Name,N Images
1,SLACS,GAL-GAL,A,SIE model,spectroscopic,spectroscopic,Luminous red galaxy lensing three background o...,1.16,0.02,0.4400,0.0001,1.192,0.001,232,39,2008-08-01,SDSS J0008-0004,3


### Coordinates table (13)

In [15]:
def coordinates(df_list):
    coords_df = pick_table(df_list, 'Coordinates:') #.drop(2, axis=1)  # Manual coordinates
    coords_df.drop(2, axis=1, inplace=True)    
    coords_df = coords_df.set_index(0).T
    coords_df['Coordinates:'] = 'Manual'
    coords_df = coords_df.set_index('Coordinates:')

    return coords_df

In [16]:
coordinates(df_list)

KeyError: '[2] not found in axis'

### External links


In [958]:
def external(df_list, filename):    
    #external_df.columns = [''] * len(external_df.columns)
    #external_df.drop('External Links:.1', axis=1, inplace=True)
    
    soup = BeautifulSoup(open(filename), "lxml")
    external_df = pick_table(df_list, 'External Links:')
    for table in soup.find_all('table'):
            
        if table.th is not None and table.th.text == 'External Links:':
            refs = get_table_links(table)
        else:
            pass
            
            #if external_df is not None:
    external_df.columns = ['database', 'link']
    external_df = external_df.set_index('database')
    drop_nans(external_df)
    external_df['links'] = refs
    external_df.drop('link', axis=1, inplace=True)
    
    return external_df

### Flux table (SDSS 16, HST 17)

In [978]:
########### outros filtrosssss CFHTLS
def sdss(df_list):
    df = pick_table(df_list,'Filter')
    if df is not None and'SDSS' in df.columns.values:
        sdss_df = df.set_index('Band')
        sdss_df = sdss_df.drop(columns=['Unnamed: 1'])
    else:
        sdss_df = pd.DataFrame()

    return sdss_df


def hst(df_list):
    hst_df = pick_table(df_list,'HST')

    if hst_df is not None:
        hst_df = hst_df.drop(columns=['Unnamed: 1'])
        hst_df = hst_df.set_index('Band')
    else:
        hst_df = pd.DataFrame()
    return hst_df


def flux(df_list):
    sdss_df = sdss(df_list)
    hst_df = hst(df_list)

    flux_df = pd.concat([sdss_df, hst_df])
    #flux_df['release'] = list(sdss_df['SDSS']) + list(hst_df['HST'])
    #flux_df = flux_df.drop(columns=['SDSS','HST'])

    drop_nans(flux_df)
    split_col(flux_df, [['Lens Magnitude', 'lens_mag'], ['Flux [nmaggie]', 'Flux (nmaggie)'],
                    ['Reff [″]', "ref (arcsec)"], ['axis ratio (AB)', 'axis_ratio (AB)'],
                    ['PA [° E of N]','PA (deg)']])
    return flux_df

In [966]:
flux(df_list)    #CFHTLS

""


### Redshift table (18, 19)

In [899]:
def redshift(df_list):

    z_lens_df = pick_table(df_list, 'Lens Plane Images:')
    if z_lens_df is not None:
        z_lens_df = z_lens_df['Lens Plane Images:']
        #z_lens_df = z_lens_df.set_index('No.')
    else:
        z_lens_df = pd.DataFrame()

    z_source_df = pick_table(df_list, 'Source Plane Images:')
    if z_source_df is not None:
        z_source_df = z_source_df['Source Plane Images:']
        #z_source_df = z_source_df.set_index('No.')
    else:
        z_source_df = pd.DataFrame()

    z_df = pd.concat([z_lens_df, z_source_df], keys=['Lens', 'Source'])
    drop_nans(z_df)

    return z_df

In [900]:
z_lens_df = pick_table(df_list, 'Lens Plane Images:')
z_lens_df

Lens Plane Images:                             
                 No. Label     z   z_err Flux Mag
0                  1   NaN  0.44  0.0001  NaN NaN

In [901]:
redshift(df_list)

/home/viviane/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



No.      z   z_err
Lens   0    1  0.440  0.0001
Source 0    1  1.192  0.0010
       1    2  1.192  0.0010
       2    3  1.192  0.0010

### Time delay table (21)

In [902]:
def time(df_list):
    time_df = pick_table(df_list, 'Time Delays:')

    if time_df is not None:
        times = list(time_df['Time Delays:']['Time Delay (days)'])
    #print(times)
        if np.isnan(times).all():
            time_df = None

    return time_df

In [903]:
d = time(df_list)
type(d)

NoneType

### References table (14)

In [904]:
def refs(filename, df_list):
    soup = BeautifulSoup(open(filename), "html.parser")

    for table in soup.find_all('table'):
    # links = [np.where(tag.has_attr('href'),tag.get('href'),"no link") for tag in tb.find_all('a')]

        links = get_table_links(table)
        #print(links)

        if len(links) >= 2 and 'citation' in links[1]:
            papers = links[1::2]

    ref_df = pick_table(df_list, 'Discovery\xa0Paper')
    ref_df = ref_df.drop('Unnamed: 0', axis=1)

    ref_df['Links'] = papers
    ref_df.columns = ['Author', 'Title', 'Discovery', 'Links']
    return ref_df

In [905]:
refs(filename, df_list)

,Author,Title,Discovery,Links
0,Cardone et al. 2011 [MNRAS416(2011)1822],Secondary infall model and dark matter scaling...,NaN,http://admin.masterlens.org/citation.php?refer...
1,Auger et al. 2009 [ApJ705(2009)1099],"The Sloan Lens ACS Survey. IX. Colors, Lensing...",NaN,http://admin.masterlens.org/citation.php?refer...
2,Newton et al. 2009 [ApJ696(2009)1125],Enhanced Lensing Rate by Clustering of Massive...,NaN,http://admin.masterlens.org/citation.php?refer...
3,Grillo et al. 2009 [A&A501(2009)461],Photometric mass and mass decomposition in ear...,NaN,http://admin.masterlens.org/citation.php?refer...
4,Treu et al. 2009 [ApJ690(2009)670],The SLACS Survey. VIII. The Relation between E...,NaN,http://admin.masterlens.org/citation.php?refer...
5,Bolton et al. 2008 [ApJ682(2008)964],The Sloan Lens ACS Survey. V. The Full ACS Str...,Discovery Paper,http://admin.masterlens.org/citation.php?refer...


### Main code

In [1002]:
for lens_n in range(1,51):
    filename = "/home/viviane/GravLens/Scraping/masterlens/masterlens_" + str(lens_n) + ".html"
    #filename = '/home/vivianemm/PycharmProjects/GravLens/Scraping/masterlens/masterlens_1.html'
    df_list = pd.read_html(filename)

    system_df = system(df_list)
    coords_df = coordinates(df_list)
    flux_df = flux(df_list)
    z_df = redshift(df_list)
    ref_df = refs(filename, df_list)
    external_df = external(df_list, filename)
    time_df = time(df_list)
    
    save(system_df, 'system_{}.csv'.format(lens_n))
    save(coords_df, 'coordinates_{}.csv'.format(lens_n))
    save(flux_df, 'flux_{}.csv'.format(lens_n), i=True)
    save(z_df, 'redshift_{}.csv'.format(lens_n), i=True)
    save(ref_df, 'references_{}.csv'.format(lens_n))
    save(external_df, 'external_{}.csv'.format(lens_n),i=True)
    
    print(lens_n)

/home/viviane/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/viviane/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [1029]:
pd.read_csv('results/coordinates_50.csv')

,RA Hrs,RA Mins,RA Secs,RA [°],Dec Degrees,Dec Arcmin,Dec Arcsec,Dec [°]
0,10,4,34,151.143143,41,12,42.8,41.212109
